### EDA

In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO 
import numpy as np

In [ ]:
import matplotlib.pyplot as plt 
import cv2

In [ ]:
COLORS = [
    (204,24,30),
    (39,147,232),
    (85,153,0),
    (102,102,102),
    (27,133,184),
    (90,82,85),
    (85,158,131),
    (174,90,65),
    (195,203,113),
    (243,119,54),
    (184,167,234)
]

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'vfnet_r2_101_fpn_mdconv_c3-c5_mstrain_2x_coco'
folder_name = 'vfnet'

# config file 들고오기
cfg = Config.fromfile(f'./configs/{folder_name}/{model_name}.py')

In [ ]:
PREFIX = '../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


cfg.seed=0
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{model_name}'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [ ]:
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_1.pth') # 모델 save path 

dataset = build_dataset(cfg.data.val)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
# model.eval()
# model(dataset[0])

In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
# output = [i[0] for i in output] # htc 같이 하나의 이미지에 예측 마스크 까지 던저준다면 실행하면 됩니다.

In [ ]:
# Felzenszwalb et al
def non_max_suppression_fast(boxes, scores, iou_threshold):
    
    '''
        boxes : coordinates of each box
        scores : score of each box
        iou_threshold : iou threshold(box with iou larger than threshold will be removed)
    '''
    
    if len(boxes) == 0:
        return []

    # Init the picked box info
    pick = []

    # Box coordinate consist of left top and right bottom
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    # Compute area of each boxes
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    # Greedily select the order of box to compare iou
    idxs = np.argsort(scores)

    while(len(idxs) > 0):
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # With vector implementation, we can calculate fast
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        intersection = w * h

        # Calculate the iou
        iou = intersection / (area[idxs[:last]] + area[idxs[last]] - intersection)

        idxs = np.delete(idxs, np.concatenate(([last], np.where(iou > iou_threshold)[0])))

    return boxes[pick].astype("int")


def draw_result(idx):
    out = output[idx]
    coco = COCO(annotation_file='../input/data/val.json')
    data_dir = '../input/data/'
    idx = coco.getImgIds(idx)
    img = cv2.imread(data_dir+coco.load_imgs(idx)[0]['file_name'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pred_img = img.copy()

    anns = coco.load_anns(coco.get_ann_ids(idx))
    for ann in anns:
        cls_id = ann['category_id']
        x,y,w,h = [int(i) for i in ann['bbox']]
        img = cv2.rectangle(img, (x,y), (x+w, y+h), COLORS[cls_id], 3)
        text = coco.cats[cls_id]['name']
        cv2.putText(img, text, (x+10,y+19), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,0))

    for c,bboxes in enumerate(out):
#         bboxes = non_max_suppression_fast(bboxes[:,:4], bboxes[:,-1], 0.00000000007)
        for bbox in bboxes:
#             print(bbox)
            x1,y1,x2,y2 = bbox[:-1].astype('int64')
            score = str(bbox[-1])[:5]
#             score = ''
            pred_img = cv2.rectangle(pred_img, (x1,y1), (x2,y2), COLORS[c], 3 )
            text = coco.cats[c]['name'] + ":" + score
            cv2.putText(pred_img, text, (x1,y1), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,0))
            print(text)


    fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(20,20))
    ax[0].imshow(img)
    ax[1].imshow(pred_img)
    plt.show()

In [ ]:
draw_result(200)

In [ ]:
draw_result(100)

In [ ]:
draw_result(0)

In [ ]:
cfg.train_pipeline

In [ ]:
for i in data_loader:
    a = i
    break

In [ ]:
img.shape

In [ ]:
from PIL import Image
for i in range(10):
    image_id = coco.getImgIds(imgIds=i)
    image_infos = coco.loadImgs(image_id)[0]
    img = cv2.imread(os.path.join("/opt/ml/input/data", image_infos["file_name"]))
    png_name = image_infos["file_name"].replace("jpg", "png")
    annIds = coco.getAnnIds(imgIds=image_infos["id"])
    annList = coco.loadAnns(annIds)
    mask = np.ones((img.shape[:2]))
    mask *= 255
    for i, ann in enumerate(annList):
        anmask = coco.annToMask(ann)
        anmask = np.int_(anmask)
        category_id = ann['category_id']
        anmask *= (category_id - (255))
        mask += anmask
        png_img = Image.fromarray(np.uint8(mask), "L")
    
#     img.close()

    png_img.save(os.path.join("/opt/ml/input/data/seg_img", png_name), "PNG")

In [ ]:
cv2.imread('../../input/data/seg_img/batch_01_vt/0002.png')

In [ ]:
import pycocotools

In [ ]:
coco.annToMask(ann['segmentation'])

In [ ]:
import glob
f_lst = glob.glob('../../input/data/seg_img_coco/*')
res = np.array([])
for f in f_lst:
    res = np.append(res, (np.unique(cv2.imread(f))))

In [ ]:
import json
with open( '../stuff_val2017.json' , 'r') as f:
    a = json.load(f)

In [ ]:
a['annotations'][0]